# Analise API

In [1]:
import requests
import pandas as pd
from collections import Counter


## 1 Quantos feriados há no Brasil em todo o ano de 2024?


In [2]:
ano = 2024
pais = 'BR'
url = f'https://date.nager.at/api/v3/PublicHolidays/{ano}/{pais}'
resposta = requests.get(url)
feriados = resposta.json()

df_feriados = pd.DataFrame(feriados)
total_feriados = len(df_feriados)
nomes_unicos = df_feriados['name'].nunique()
print(f'Total de dias de feriados em {ano}: {total_feriados}')
print(f'Número de feriados com nomes únicos: {nomes_unicos}')

Total de dias de feriados em 2024: 15
Número de feriados com nomes únicos: 14


Vamos observar quais, já que é um número inspecionável e notar que o Carnaval é um feriado de dois dias, ao contrário dos demais, então temos mais dias de feriado (15) do que feriados (14).

In [3]:
df_feriados

,date,localName,name,countryCode,fixed,global,counties,launchYear,types
0,2024-01-01,Confraternização Universal,New Year's Day,BR,False,True,None,None,[Public]
1,2024-02-12,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
2,2024-02-13,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
3,2024-03-29,Sexta-feira Santa,Good Friday,BR,False,True,None,None,[Public]
4,2024-03-31,Domingo de Páscoa,Easter Sunday,BR,False,True,None,None,[Public]
5,2024-04-21,Dia de Tiradentes,Tiradentes,BR,False,True,None,None,[Public]
6,2024-05-01,Dia do Trabalhador,Labour Day,BR,False,True,None,None,[Public]
7,2024-05-30,Corpus Christi,Corpus Christi,BR,False,True,None,None,[Public]
8,2024-07-09,Revolução Constitucionalista de 1932,Constitutionalist Revolution of 1932,BR,False,False,[BR-SP],None,[Public]
9,2024-09-07,Dia da Independência,Independence Day,BR,False,True,None,None,[Public]


## 2 Qual mês de 2024 tem o maior número de feriados?

In [4]:
df_feriados['date'] = pd.to_datetime(df_feriados['date'])
df_feriados['mes'] = df_feriados['date'].dt.month
contagem_meses = df_feriados['mes'].value_counts()
mes_mais_feriados = contagem_meses.idxmax()
print(f'Mês com mais feriados: {mes_mais_feriados} ({contagem_meses[mes_mais_feriados]} feriados)')

Mês com mais feriados: 11 (3 feriados)


## 3 Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?


In [5]:
df_feriados['dia_semana'] = df_feriados['date'].dt.weekday  # 0=segunda-feira, 6=domingo
dias_uteis = df_feriados[df_feriados['dia_semana'] < 5]
total_dias_uteis = len(dias_uteis)
print(f'Feriados em dias de semana: {total_dias_uteis}')

Feriados em dias de semana: 10


## 4 Qual foi a temperatura média em cada mês de 01/01/2024 a 01/08/2024 no Rio de Janeiro?

Pode ser que a pergunta tenha um erro de digitação e queira até 31/08/2024, neste caso, basta alterar o fim.

In [6]:
#Para as coordenadas da Cidade do Rio de Janeiro
latitude = -22.9068
longitude = -43.1729
inicio = '2024-01-01'
fim = '2024-08-01'
url = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={inicio}&end_date={fim}&daily=temperature_2m_max,temperature_2m_min&timezone=America%2FSao_Paulo'
resposta = requests.get(url)
dados = resposta.json()

temperaturas = {
    'data': dados['daily']['time'],
    'temp_max': dados['daily']['temperature_2m_max'],
    'temp_min': dados['daily']['temperature_2m_min']
}


In [7]:
df_temperaturas = pd.DataFrame(temperaturas)
df_temperaturas['data'] = pd.to_datetime(df_temperaturas['data'])
df_temperaturas['temp_media'] = (df_temperaturas['temp_max'] + df_temperaturas['temp_min']) / 2
df_temperaturas['mes'] = df_temperaturas['data'].dt.month
medias_mensais = df_temperaturas.groupby('mes')['temp_media'].mean()

medias_mensais

,temp_media
mes,
1,27.104839
2,27.722414
3,26.891935
4,25.650000
5,25.583871
6,23.236667
7,21.674194
8,21.800000


Aqui, além de responder a pergunta principal, vamos deixar guardado o dataframe de temperaturas diárias do período, com grandes chances de ser útil novamente.

In [8]:
df_temperaturas.head()

,data,temp_max,temp_min,temp_media,mes
0,2024-01-01,27.9,21.8,24.85,1
1,2024-01-02,29.8,23.0,26.40,1
2,2024-01-03,30.2,23.2,26.70,1
3,2024-01-04,28.8,23.5,26.15,1
4,2024-01-05,28.1,23.2,25.65,1


## 5 Qual foi o tempo predominante em cada mês nesse período?

In [9]:
# URL do JSON com as descrições dos códigos de tempo
url = 'https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json'

resposta = requests.get(url)
dados_json = resposta.json()

# Transformar o JSON em um DataFrame
df_descricao = pd.DataFrame.from_dict(dados_json, orient='index')

# Extrair as descrições diurnas e associá-las aos códigos
mapeamento_codigos = {int(codigo): info['day']['description'] for codigo, info in dados_json.items()}

In [40]:
url = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={inicio}&end_date={fim}&daily=weathercode&timezone=America%2FSao_Paulo'
resposta = requests.get(url)
dados = resposta.json()

condicoes = {
    'data': dados['daily']['time'],
    'codigo_tempo': dados['daily']['weathercode']
}
df_condicoes = pd.DataFrame(condicoes)
df_condicoes['data'] = pd.to_datetime(df_condicoes['data'])
df_condicoes['mes'] = df_condicoes['data'].dt.month

predominantes = {}
for mes, grupo in df_condicoes.groupby('mes'):
    contagem = Counter(grupo['codigo_tempo'])
    codigo_predominante = contagem.most_common(1)[0][0]
    predominantes[mes] = mapeamento_codigos.get(codigo_predominante)

predominantes

{1: 'Rain',
 2: 'Light Drizzle',
 3: 'Light Drizzle',
 4: 'Cloudy',
 5: 'Light Drizzle',
 6: 'Cloudy',
 7: 'Cloudy',
 8: 'Cloudy'}

## 6 Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

A estratégia básica aqui é aproveitar os dataframes que já criamos e fazer um Join. Neste caso vamos usar a intersecção para restringir ao período específico. Note que por mais que seja uma ação pequena, damos preferência de performance para manipular o codigo_tempo e só mapear na string tempo ao final.

In [29]:
df_feriados_temperatura = pd.merge(df_feriados[['date','localName']]
                                   , df_temperaturas
                                   , left_on='date', right_on='data', how='inner')

df_feriados_temperatura = pd.merge(df_feriados_temperatura[['data','localName','temp_media']]
                                   , df_condicoes
                                   , left_on='data', right_on='data', how='inner')

df_feriados_temperatura['tempo'] = df_feriados_temperatura['codigo_tempo'].map(mapeamento_codigos)
df_feriados_temperatura.drop(columns=['mes','codigo_tempo'], inplace=True)
df_feriados_temperatura

,data,localName,temp_media,tempo
0,2024-01-01,Confraternização Universal,24.85,Light Drizzle
1,2024-02-12,Carnaval,30.80,Cloudy
2,2024-02-13,Carnaval,31.15,Light Drizzle
3,2024-03-29,Sexta-feira Santa,25.70,Light Rain
4,2024-03-31,Domingo de Páscoa,24.70,Drizzle
5,2024-04-21,Dia de Tiradentes,23.50,Partly Cloudy
6,2024-05-01,Dia do Trabalhador,28.35,Sunny
7,2024-05-30,Corpus Christi,20.90,Light Drizzle
8,2024-07-09,Revolução Constitucionalista de 1932,21.95,Drizzle


## 7 Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

A princípio, poderíamos alterar a data fim do df_condicoes gerado na questão 5  e aproveitar a mesma lógica da questão 6 de fazer o Join das 3 tabelas. Porém por uma questão de mostrar outras opções, vamos iterar na seguinte tabela que está parcialmente preenchida pelas questões anteriores e completar com requisições específicas. <p>
Note que este método é para ser instrutivo, pois neste caso particular, abrir 15 requisições é até mais lento do que buscar o ano completo com uma conexão como nas questões anteriores.

Então vamos ver o que temos e completar:

In [30]:
df_todos_feriados_temperatura = pd.merge(df_feriados[['date','localName']], df_temperaturas, left_on='date', right_on='data', how='left')
df_todos_feriados_temperatura

,date,localName,data,temp_max,temp_min,temp_media,mes
0,2024-01-01,Confraternização Universal,2024-01-01,27.9,21.8,24.85,1.0
1,2024-02-12,Carnaval,2024-02-12,36.6,25.0,30.80,2.0
2,2024-02-13,Carnaval,2024-02-13,37.5,24.8,31.15,2.0
3,2024-03-29,Sexta-feira Santa,2024-03-29,29.1,22.3,25.70,3.0
4,2024-03-31,Domingo de Páscoa,2024-03-31,27.5,21.9,24.70,3.0
5,2024-04-21,Dia de Tiradentes,2024-04-21,28.4,18.6,23.50,4.0
6,2024-05-01,Dia do Trabalhador,2024-05-01,34.7,22.0,28.35,5.0
7,2024-05-30,Corpus Christi,2024-05-30,23.5,18.3,20.90,5.0
8,2024-07-09,Revolução Constitucionalista de 1932,2024-07-09,23.9,20.0,21.95,7.0
9,2024-09-07,Dia da Independência,NaT,NaN,NaN,NaN,NaN


In [41]:
for index, feriado in df_todos_feriados_temperatura.iterrows():
    data = feriado['date']
    if data.strftime('%Y-%m-%d') >= fim:
        url = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={data.strftime("%Y-%m-%d")}&end_date={data.strftime("%Y-%m-%d")}&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=America%2FSao_Paulo'
        resposta = requests.get(url)
        dados = resposta.json()
        temp_max = dados['daily']['temperature_2m_max'][0]
        temp_min = dados['daily']['temperature_2m_min'][0]
        temp_media = (temp_max + temp_min) / 2
        codigo_tempo = dados['daily']['weathercode'][0]

        # Atualiza o DataFrame diretamente usando o índice
        df_todos_feriados_temperatura.at[index, 'temp_max'] = temp_max
        df_todos_feriados_temperatura.at[index, 'temp_min'] = temp_min
        df_todos_feriados_temperatura.at[index, 'temp_media'] = temp_media
        df_todos_feriados_temperatura.at[index, 'codigo_tempo'] = codigo_tempo
        df_todos_feriados_temperatura.at[index,'tempo'] = mapeamento_codigos.get(codigo_tempo)
    else:
        url = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={data.strftime("%Y-%m-%d")}&end_date={data.strftime("%Y-%m-%d")}&daily=weathercode&timezone=America%2FSao_Paulo'
        resposta = requests.get(url)
        dados = resposta.json()
        codigo_tempo = dados['daily']['weathercode'][0]

        # Atualiza o DataFrame diretamente usando o índice
        df_todos_feriados_temperatura.at[index, 'codigo_tempo'] = codigo_tempo
        df_todos_feriados_temperatura.at[index,'tempo'] = mapeamento_codigos.get(codigo_tempo)


In [42]:
df_todos_feriados_temperatura[['date','localName','temp_media','tempo','codigo_tempo']]

,date,localName,temp_media,tempo,codigo_tempo
0,2024-01-01,Confraternização Universal,24.85,Light Drizzle,51.0
1,2024-02-12,Carnaval,30.80,Cloudy,3.0
2,2024-02-13,Carnaval,31.15,Light Drizzle,51.0
3,2024-03-29,Sexta-feira Santa,25.70,Light Rain,61.0
4,2024-03-31,Domingo de Páscoa,24.70,Drizzle,53.0
5,2024-04-21,Dia de Tiradentes,23.50,Partly Cloudy,2.0
6,2024-05-01,Dia do Trabalhador,28.35,Sunny,0.0
7,2024-05-30,Corpus Christi,20.90,Light Drizzle,51.0
8,2024-07-09,Revolução Constitucionalista de 1932,21.95,Drizzle,53.0
9,2024-09-07,Dia da Independência,24.90,Cloudy,3.0


In [56]:
frio = df_todos_feriados_temperatura['temp_media'] < 20
sol = df_todos_feriados_temperatura['codigo_tempo'] <= 2
nublado = df_todos_feriados_temperatura['codigo_tempo'] == 3
chuvisco_leve = df_todos_feriados_temperatura['codigo_tempo'] == 51
chuvisco = df_todos_feriados_temperatura['codigo_tempo'] == 53
chuvosos = df_todos_feriados_temperatura['codigo_tempo'] >= 55

# Combinando as condições para determinar se é um dia de praia
praia = ~frio & ~nublado & ~chuvosos
praia_sol = ~frio & sol

# Filtrando os feriados "não aproveitáveis"
nao_aproveitaveis = frio | chuvosos | nublado
questionavel = ~frio & (chuvisco | chuvisco_leve)

df_nao_aproveitaveis = df_todos_feriados_temperatura[nao_aproveitaveis]
df_nao_aproveitaveis[['date','localName','temp_media','tempo','codigo_tempo']]

,date,localName,temp_media,tempo,codigo_tempo
1,2024-02-12,Carnaval,30.80,Cloudy,3.0
3,2024-03-29,Sexta-feira Santa,25.70,Light Rain,61.0
9,2024-09-07,Dia da Independência,24.90,Cloudy,3.0
10,2024-10-12,Nossa Senhora Aparecida,27.80,Rain,63.0
13,2024-11-20,Dia da Consciência Negra,25.60,Cloudy,3.0
14,2024-12-25,Natal,27.55,Cloudy,3.0


In [57]:
df_todos_feriados_temperatura[questionavel]

,date,localName,data,temp_max,temp_min,temp_media,mes,NaN,codigo_tempo,tempo
0,2024-01-01,Confraternização Universal,2024-01-01,27.9,21.8,24.85,1.0,22.1,51.0,Light Drizzle
2,2024-02-13,Carnaval,2024-02-13,37.5,24.8,31.15,2.0,22.1,51.0,Light Drizzle
4,2024-03-31,Domingo de Páscoa,2024-03-31,27.5,21.9,24.70,3.0,22.1,53.0,Drizzle
7,2024-05-30,Corpus Christi,2024-05-30,23.5,18.3,20.90,5.0,22.1,51.0,Light Drizzle
8,2024-07-09,Revolução Constitucionalista de 1932,2024-07-09,23.9,20.0,21.95,7.0,22.1,53.0,Drizzle
11,2024-11-02,Dia de Finados,NaT,30.0,20.3,25.15,NaN,22.1,51.0,Light Drizzle
12,2024-11-15,Proclamação da República,NaT,27.1,20.8,23.95,NaN,22.1,53.0,Drizzle


Aqui cabe notar que preferi separar os dias certamente não aproveitáveis, dos questionáveis. Pelos hábitos cariocas, um dia com chuvisco poderia não ser bem recebido, embora não seja o mesmo peso da chuva do dia chuvoso. <P>
Essa é uma decisão importante, pois permite reclassificar rapidamente os questionáveis em um grupo correto, caso se redefinam pelas regras de negócio.

## 8 Qual foi o feriado "mais aproveitável" de 2024?

In [58]:
df_todos_feriados_temperatura[praia_sol]

,date,localName,data,temp_max,temp_min,temp_media,mes,NaN,codigo_tempo,tempo
5,2024-04-21,Dia de Tiradentes,2024-04-21,28.4,18.6,23.50,4.0,22.1,2.0,Partly Cloudy
6,2024-05-01,Dia do Trabalhador,2024-05-01,34.7,22.0,28.35,5.0,22.1,0.0,Sunny


Como só temos 2 candidatos, não chega a ser necessário ordená-los, mas poderíamos ordenar por ordem decrescente de temperatura média e crescente de codigo do tempo.

Assim, nosso feriado mais aproveitável foi o Dia do Trabalhador. Merecido :)

## 9 Sugestões e ideias
Além do que pode entrar como recurso visual, temos alguns dados que são interessantes de se obter já com o que temos. <p>
Médias moveis, por exemplo, como a temperatura estava ao redor dos eventos, eventos satélites, emendas de feriado, ou o chamado feriadão prolongado.

Pequenos modelos de predição com scikit também podem revelar quando a temperatura mudou subtamente da tendência e pegou a população mais desprevenida.

Grandes oscilações de temperatura intradia e diária também são fatores importantes para o conforto e até segurança da população.